In [7]:
import pandas as pd

tickets= pd.read_csv(r'C:\Users\Scc\Downloads\data_safe_copy.csv')
print (len(tickets))
print(tickets.head())
tickets.info()

500
  ticket_id         opened_at   category priority resolution_minutes
0   TK-0001  2024-03-01 09:00    billing      low                 45
1   TK-0002  2024-03-01 09:30    account   medium                 30
2   TK-0003  2024-03-01 10:00  technical     high                 75
3   TK-0004  2024-03-01 10:30    billing      low                NaN
4   TK-0005  2024-03-01 11:00    account   medium            unknown
<class 'pandas.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   ticket_id           500 non-null    str  
 1   opened_at           500 non-null    str  
 2   category            500 non-null    str  
 3   priority            500 non-null    str  
 4   resolution_minutes  450 non-null    str  
dtypes: str(5)
memory usage: 19.7 KB


In [9]:
tickets["opened_at"]= pd.to_datetime(tickets["opened_at"],errors="coerce")
tickets["category"] = tickets["category"].str.strip().str.lower()
tickets["priority"] = tickets["priority"].str.strip().str.lower()
print(tickets["category"].unique())
print(tickets["priority"].unique())

<StringArray>
['billing', 'account', 'technical']
Length: 3, dtype: str
<StringArray>
['low', 'medium', 'high']
Length: 3, dtype: str


In [12]:
invalid_markers = ["unknown", "n/a", "??"]
intentional_issue_count = tickets["resolution_minutes"] \
    .astype(str).str.lower().isin(invalid_markers).sum()
print(f"Intentionally introduced issues: {intentional_issue_count}")

tickets["resolution_minutes"]= pd.to_numeric(tickets["resolution_minutes"],errors="coerce")
missing_count = tickets["resolution_minutes"].isna().sum()
print(missing_count)

Intentionally introduced issues: 0
100


In [18]:
before_count = len(tickets)
tickets_clean = tickets.dropna(subset=["resolution_minutes"]).copy()
after_count = len(tickets_clean)
non_negative_check = (tickets_clean["resolution_minutes"] >= 0).all()
category_normalized_check = (
    tickets_clean["category"]
    .eq(tickets_clean["category"].str.strip().str.lower())
    .all())
priority_normalized_check = (
    tickets_clean["priority"]
    .eq(tickets_clean["priority"].str.strip().str.lower())
    .all())


print(before_count)
print(after_count)

print(non_negative_check)
print(category_normalized_check)
print(priority_normalized_check)

500
400
True
True
True


In [19]:
summary = {
    "total_records": len(tickets),
    "clean_records": len(tickets_clean),
    "missing_resolution": tickets["resolution_minutes"].isna().sum(),
    "avg_resolution": tickets_clean["resolution_minutes"].mean(),
    "max_resolution": tickets_clean["resolution_minutes"].max(),
}

# Print summary
print(summary)

# Integrity check
assert (
    summary["clean_records"] + summary["missing_resolution"]
    == summary["total_records"]
), "Record counts do not add up!"

{'total_records': 500, 'clean_records': 400, 'missing_resolution': np.int64(100), 'avg_resolution': np.float64(57.5), 'max_resolution': np.float64(120.0)}
